In [1]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [2]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#     add_from_embedded = {}
  
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
  
track_loop_sets = [
'9b1063f5-c891-44a8-b44a-3b4879257366',
'c5db8085-92bd-4d06-a22c-7958e4a78e19',]

dekker_mc = [
'51d9ac15-e55e-4f64-85e5-01998e349cf1',
'998573fb-384e-4792-a621-254d705d4ec9',
]

gianluca = [
'8e9cfe23-aac5-4e66-9514-e880fe34cbc4',
'e4510669-23bb-45cb-a6a0-f6bb0cf75dd0',
'1c75ed00-ba21-4484-b510-f0875d814bc7',
'3ebd99e8-db53-44e9-a831-3e4813d2dd99',
'906bd763-6439-4445-9e4a-5f814b246070',
]

kaufman_sets = [
'c85de504-96cf-4856-b7f3-398e6bf1e37b',
'c50006d4-a98d-4366-bd5b-650fee1696ee',
'43586c61-a527-43d3-b95a-1bc063fa1fb7',
'e2b336fb-dda9-4eff-9125-0603cc99916e',
'f41f9b4b-9f13-4e25-abed-13fcf99b5a73',
'1ed849f5-cf49-4190-84a9-161f13906c21',
'c33e4a8d-cd5e-4bf2-8973-88a46efd2173',
'd5fd5f93-8ffc-4bdc-8db9-1457d119248b',
]

dekker_dil = ['233740be-ffbe-4acc-b6a4-24bbee96ca73'
]

sprite_sets = ['cc82a2af-95e7-4447-b571-87d4785b33ec']

my_exp = sprite_sets[0]

print my_exp
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




cc82a2af-95e7-4447-b571-87d4785b33ec
workflow 2324ad76-ff37-4157-8bcc-3ce72b7dace9 released to project
workflow d3f25cd3-e726-4b3c-a022-48f844474b41 released to project
177
177


In [3]:
file_fastq = [i['uuid'] for i in store['file_fastq']]
print(file_fastq)

file_proc = [i['uuid'] for i in store['file_processed']]
print(file_proc)

file_ref = [i['status'] for i in store['file_reference']]
print(file_ref)

for a_type in store:
    print a_type

[u'8d290819-fd53-4b7d-a55f-af1ce30fe45d', u'd8c52f6e-3428-4faa-8fb5-2aa79953d038', u'47b20e1f-b2a9-4a62-bdce-b6fcc1e55c97', u'7127a439-a103-447c-b7fb-50ca392dac67', u'2e4cd2e4-f1ae-44cc-abb6-d8c1eadef259', u'cfdb7fcc-c8dd-473e-abb1-6ee354bffffc', u'9ada472a-69cc-49ff-a4d2-eb75dde45988', u'ca82d5e3-7bd1-46e0-a893-b0849524e1f9', u'a06b0d36-9363-4c20-b103-3217f937a182', u'15e76ad6-a0f9-4d99-86bd-3a0c6c70c140', u'd77f30e0-63d1-40a4-9d01-5dc563ec0ae8', u'809a83e2-bdf1-4f30-b2cd-fc4c4d3c43e2', u'63820312-e8d4-485e-a014-d8df4a04dc75', u'3a7fe2cf-f965-4722-af84-ac312ded2b0e', u'165c17f7-4e58-464a-a065-e728464c62bd', u'55bab5b0-3408-4d0c-9d34-e4cffd95a4f3', u'a0a648ed-02b2-40d3-a8b1-f3d2a135a9d2', u'df679fa8-e2e4-492a-8233-8472986efd06', u'5f5c82c1-a4d5-43e4-a652-105ebc9ec872', u'a3840b7c-59af-424d-86e1-dcaa093c27c3', u'7b646365-1a5e-4ed0-bf39-520b1e812a22', u'1024da6a-73b7-41cc-a01e-60f3e022599c', u'a65b2920-9dd7-4a26-b0f6-0081d5a35cf5', u'31605b7f-0ba6-4472-b2bc-8e219c83705f', u'4afff87b-bc9a

KeyError: 'file_processed'

In [4]:
## Check which items would be released to project
counter = 0
for a_type in store:

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in [ 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
print counter

experiment_set_replicate submission in progress cc82a2af-95e7-4447-b571-87d4785b33ec
quality_metric_fastqc submission in progress 13322c4a-b981-42db-9417-daa4e6bbbdc4
quality_metric_fastqc submission in progress 00ecdc4b-83ec-4811-871d-37e6e7610422
quality_metric_fastqc submission in progress eff0a4ea-516c-46e0-aff1-a798bd38fa53
quality_metric_fastqc submission in progress a81538d1-3a1c-43f0-9527-f5bcac72c39a
quality_metric_fastqc in review by lab 3fd9ec54-92ef-408a-a872-33758c44c0ce
quality_metric_fastqc submission in progress 9e73c93f-a120-44cf-88f7-55fbb3ad28e0
quality_metric_fastqc in review by lab 9925718c-5b50-4f65-8571-eae50bb3f5d8
quality_metric_fastqc submission in progress 733c8539-1ed4-4788-a5dd-9f4b2b4d55c6
quality_metric_fastqc submission in progress e39406d3-f627-4b52-9fdd-a2f61a185ee4
quality_metric_fastqc submission in progress d4065b06-dbf0-45d3-ad67-9155cb9995f4
quality_metric_fastqc submission in progress dde1d208-7680-46ca-8fdc-8e90463c3b5e
quality_metric_fastqc in 

In [5]:
# release to project
counter = 0
for a_type in store:
    rel_type = 0
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released to project"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)
            rel_type += 1
    if rel_type > 0:
        print(a_type, rel_type)

print counter

experiment_set_replicate submission in progress cc82a2af-95e7-4447-b571-87d4785b33ec
(u'experiment_set_replicate', 1)
quality_metric_fastqc submission in progress 13322c4a-b981-42db-9417-daa4e6bbbdc4
quality_metric_fastqc submission in progress 00ecdc4b-83ec-4811-871d-37e6e7610422
quality_metric_fastqc submission in progress eff0a4ea-516c-46e0-aff1-a798bd38fa53
quality_metric_fastqc submission in progress a81538d1-3a1c-43f0-9527-f5bcac72c39a
quality_metric_fastqc in review by lab 3fd9ec54-92ef-408a-a872-33758c44c0ce
quality_metric_fastqc submission in progress 9e73c93f-a120-44cf-88f7-55fbb3ad28e0
quality_metric_fastqc in review by lab 9925718c-5b50-4f65-8571-eae50bb3f5d8
quality_metric_fastqc submission in progress 733c8539-1ed4-4788-a5dd-9f4b2b4d55c6
quality_metric_fastqc submission in progress e39406d3-f627-4b52-9fdd-a2f61a185ee4
quality_metric_fastqc submission in progress d4065b06-dbf0-45d3-ad67-9155cb9995f4
quality_metric_fastqc submission in progress dde1d208-7680-46ca-8fdc-8e904

workflow_run_awsem submission in progress 2be95601-5eb0-4881-9014-670c36d54ea6
workflow_run_awsem submission in progress 6a097d51-3146-4167-9579-6f45c0e45792
workflow_run_awsem submission in progress 5f513d7f-a8e7-4f61-bde4-09c006eeb6f0
workflow_run_awsem submission in progress 849f0577-3de6-45b8-94c2-9f9ed14bfd0f
workflow_run_awsem submission in progress 055edbc0-3be2-4628-a728-3c99c141c28f
workflow_run_awsem submission in progress b32cee32-1996-48fe-8181-9445011fa923
workflow_run_awsem submission in progress 8c8274e8-6fa8-41d8-a409-6775807d3ab5
workflow_run_awsem submission in progress 8069877c-cbfe-4ebe-a2db-ab425bd67e34
workflow_run_awsem submission in progress 40d775aa-379b-4b1b-b47f-c150d2524088
workflow_run_awsem submission in progress f6d12e9a-4af3-4a55-b558-3a9c0f620c7b
workflow_run_awsem submission in progress 3745f125-10b3-4e31-82ff-8270baa7d7c0
workflow_run_awsem submission in progress f0927b4c-59cf-4a89-817f-9776971b10c2
workflow_run_awsem submission in progress f425ada1-e